# Time Series Analysis

Time series analysis is the statistical analysis of data that is collected at regular intervals over time. It is used to understand the patterns in the data, forecast future values, and identify outliers.

### Benefits of time series analysis
- It can help to identify patterns and trends in data that would not be visible otherwise.
- It can be used to forecast future values of the data, which can be used to make better decisions.
- It can help to identify outliers and anomalies, which can be investigated further.


### Where is time series analysis used?
- In `Finance`, to forecast stock prices, currency exchange rates, and other financial market data.
- In `Retail`, to forecast sales, demand, and inventory levels.

# There are several categories in the Superstore sales data, we start from time series analysis and forecasting for furniture sales.

### Import all necessary Libraries

In [5]:
# Ignore warning related to pandas_profiling
import warnings

# advanced tools for creating and working with iterators
import itertools

# Fundamental package for linear algebra and multidimensional arrays
import numpy as np

# Python package for data manipulation and analysis
import pandas as pd

# Library for Visualization
import matplotlib

# Plotting library with Matplotlib for Python
import matplotlib.pyplot as plt

# Python module that provides classes and functions for the estimation of many different statistical models
import statsmodels.api as sm

# Use FiveThirtyEight style plots
plt.style.use("fivethirtyeight")
# Suppress all warnings
warnings.filterwarnings("ignore")

# Format the size of x and y axis tick labels
matplotlib.rcParams["axes.labelsize"] = 14
# Format the size of the x axis tick labels
matplotlib.rcParams["xtick.labelsize"] = 12
# Format the size of the y axis tick labels
matplotlib.rcParams["ytick.labelsize"] = 12
# Change the color of the text to black
matplotlib.rcParams["text.color"] = "k"

#### Let's read our dataset

In [2]:
# Let's read our data and store it in a dataframe called df
df = pd.read_excel("../Time_Series_Analysis/dataset/Sample - Superstore.xls")


In [6]:
# Let's see what's inside our data
df.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


We ahve many category in our dataset but for this analysis, we will analyze only furniture sales. So let's filter our dataset for furniture sales only.

In [9]:
# Let's create a dataframe called furniture which contains only the furniture category

furniture = df.loc[df["Category"] == "Furniture"]

# Let's print the first 5 rows of furniture

furniture.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
5,6,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694
10,11,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-TA-10001539,Furniture,Tables,Chromcraft Rectangular Conference Tables,1706.1840,9,0.20,85.3092
